<img src="https://github.com/sobester/ADRpy/raw/master/docs/ADRpy/ADRpy_splash.png" width="40%" align="left">

*The design of the primary structure of an airframe begins with establishing the air loads it will experience in service. These vary mainly with airspeed and the amount of lift demanded of the lifting surfaces throughout the range of manoeuvres flown by the aircraft.*

*The airspeed versus load factor (V-n) diagram is a concise representation of the closed set of boundaries that surround all the loading conditions the aircraft is expected to withstand. The algorithm for drawing up this **flight envelope** is usually set out by the certification document that defines the initial airworthiness requirements. In this notebook we base our discussion of the flight envelope (and how to draw it up in ADRpy) on the algorithm included in EASA's CS-23.*

# The Flight Envelope of a Fixed Wing Aircraft

For the purposes certifying the airworthiness of an aircraft from the perspective of its structural strength, the 'flight envelope' is generally depicted as a closed set of boundaries in the V (equivalent airspeed) - n (load factor) space. The pictorial representation of this area, whithin which compliance with structural strength requirements has to be demonstrated as part of the certification process, is referred to as the **V-n diagram** of the aircraft.

The V-n diagram specifies the limit loads due to symmetrical manoeuvres and gust loads (which the airframe has to be able to withstand at any combination of equivalent airspeed and load factor within the flight envelope); other structural loads (landing, bird strike, etc.) must be dealt with separately.

In this notebook we consider the construction of V-n diagrams for 'Normal category' aeroplanes (EASA defines these as aeroplanes with a passenger seating configuration of 19 or less and a maximum certified take-off mass of 8,618kg, see CS-23.2005). Specifically, the basis of the discussion below is **CS-23** (Certification Specifications for Normal- Category Aeroplanes) **Amendment 5** / the Acceptable Means of Compliance and Guidance Material to the Certification Specifications for Normal-Category Aeroplanes (**AMC & GM to CS-23**, Issue 3). 

Let us begin with an illustrative example, which we shall use to a) demonstrate how to build a CS-23 V-n diagram in ADRpy and b) as the basis for an explanation of the anatomy of a V-n diagram.

## Illustrative example

First, we import the necessary ADRpy modules. The V-n diagram generator lives in the `airworthiness` module; we'll also need code for unit conversions and for modeling the atmosphere:

In [1]:
from ADRpy import airworthiness as aw
from ADRpy import unitconversions as co
from ADRpy import atmospheres as at

Next, we define the inputs of the actual example. In *General Aviation Aircraft Design: Applied Methods and Procedures*, Snorri Gudmundsson puts forward a test case, a hypothetical aircraft with lightly loaded wings, designed to stretch an engineer's ability to produce more 'interesting' V-n diagrams. We will use this here and we define it through a set of dictionaries, which we use to instantiate an object of the `CertificationSpecifications` class.

`CertificationSpecifications` was designed to be broadly analogous to the `AircraftConcept` class (to make it easier to do both constraint analysis and V-n calculations on a tentative concept), so we'll have a similar set of dictionaries as inputs, though some can be left blank if, like here, we only do the V-n analysis.

To begin with, we don't need a design brief here, so we'll leave it blank:

In [2]:
designbrief = {}

Next, basic geometry and the maximum take-off weight in the design definition dictionary entry:

In [3]:
designdef = {'aspectratio': 11.1, 'wingarea_m2': 12.1, 'weight_n': 5872}

Key aerodynamic performance figures expressing the ability of the aircraft to generate lift:

In [4]:
designperf = {'CLmaxclean': 1.45, 'CLminclean': -1, 'CLslope': 6.28}

Next, we specify the type of propulsion system. This is not specifically needed for the V-n diagram here, it is required simply for consistency with other classes and to support features included in later releases.

In [5]:
designpropulsion = "piston"

We set the atmosphere we want to work on, by instanting an object of the `Atmosphere` class. For consistency with Gudmundsson's example, let's simply have a zero offset ISA here.

In [6]:
designatm = at.Atmosphere()

Now onto the `csbrief` dictionary.

In addition to the target cruise speed, we need to specify the **dive speed** $V_D$ (in KEAS, or 'knots equivalent airspeed'), the speed which, when exceeded during a dive, may result in buffeting, excessive vibration, damaging flutter or loss of control authority.

Should dive speed not be specified or be outside of the regulatory requirements, $V_D$ defaults to the minimum possible value as permitted by CS-23.

We also need to state the flight conditions where we want to draw up the diagram, specifically the altitude and the fraction of the maximum take-off weight (as defined in `designdef`'s `weight_n` entry) where we want to generate the diagram (diagrams are typically generated for several combinations of these figures). We'll use sea level and the full maximum take-off weight here.

We also need to pick a category (`certcat`), as defined by CS-23. We will use 'normal' in this case, designated by the category identifier `'norm'`; the other categories are 'utility' (`'util'`), 'commuter' (`'comm'`), and 'aerobatic' (`'aero'`). The magnitude of the maneuvering loads (CS-23.337) depends on this choice - for example, aerobatic aircraft must be designed for a positive limit load factor of 6, whereas the maneuvering *n* need not be greater than 3.8 in the 'normal' and 'commuter' categories.

In [7]:
csbrief={'cruisespeed_keas': 107, 'divespeed_keas': 150,
         'altitude_m': 0,
         'weightfraction': 1, 'certcat': 'norm'}

We are now ready to instantiate a `CertificationSpecifications` object with these dictionaries:

In [8]:
concept = aw.CertificationSpecifications(designbrief, designdef, designperf, designatm, designpropulsion, csbrief)

And now, for the main event. We plot the flight envelope for this aircraft using the `flightenvelope` method included in the`CertificationSpecifications` class. It takes some basic sizing parameters and generates the diagram. It returns a dictionary containing the coordinates of all the labelled points. These are also listed in the legend of the diagram.

In [9]:
points = concept.flightenvelope(textsize=15, figsize_in=[15, 10], show=True)

AttributeError: 'AircraftConcept' object has no attribute 'clmaxclean'

## Anatomy of the V-n Diagram

A V-n diagram, such as this one, is actually two diagrams on top of one another. The effects of manoeuvre loads ÃƒÆ’Ã†â€™Ãƒâ€ Ã¢â‚¬â„¢ÃƒÆ’Ã¢â‚¬Â ÃƒÂ¢Ã¢â€šÂ¬Ã¢â€žÂ¢ÃƒÆ’Ã†â€™ÃƒÂ¢Ã¢â€šÂ¬Ã…Â¡ÃƒÆ’Ã¢â‚¬Å¡Ãƒâ€šÃ‚Â¢ÃƒÆ’Ã†â€™Ãƒâ€ Ã¢â‚¬â„¢ÃƒÆ’Ã¢â‚¬Å¡Ãƒâ€šÃ‚Â¢ÃƒÆ’Ã†â€™Ãƒâ€šÃ‚Â¢ÃƒÆ’Ã‚Â¢ÃƒÂ¢Ã¢â€šÂ¬Ã…Â¡Ãƒâ€šÃ‚Â¬ÃƒÆ’Ã¢â‚¬Â¦Ãƒâ€šÃ‚Â¡ÃƒÆ’Ã†â€™ÃƒÂ¢Ã¢â€šÂ¬Ã…Â¡ÃƒÆ’Ã¢â‚¬Å¡Ãƒâ€šÃ‚Â¬ÃƒÆ’Ã†â€™Ãƒâ€ Ã¢â‚¬â„¢ÃƒÆ’Ã¢â‚¬Å¡Ãƒâ€šÃ‚Â¢ÃƒÆ’Ã†â€™Ãƒâ€šÃ‚Â¢ÃƒÆ’Ã‚Â¢ÃƒÂ¢Ã¢â‚¬Å¡Ã‚Â¬Ãƒâ€¦Ã‚Â¡ÃƒÆ’Ã¢â‚¬Å¡Ãƒâ€šÃ‚Â¬ÃƒÆ’Ã†â€™ÃƒÂ¢Ã¢â€šÂ¬Ã‚Â¦ÃƒÆ’Ã‚Â¢ÃƒÂ¢Ã¢â‚¬Å¡Ã‚Â¬Ãƒâ€¦Ã¢â‚¬Å“ or, the manoeuvre envelope, yellow dashed lines ÃƒÆ’Ã†â€™Ãƒâ€ Ã¢â‚¬â„¢ÃƒÆ’Ã¢â‚¬Â ÃƒÂ¢Ã¢â€šÂ¬Ã¢â€žÂ¢ÃƒÆ’Ã†â€™ÃƒÂ¢Ã¢â€šÂ¬Ã…Â¡ÃƒÆ’Ã¢â‚¬Å¡Ãƒâ€šÃ‚Â¢ÃƒÆ’Ã†â€™Ãƒâ€ Ã¢â‚¬â„¢ÃƒÆ’Ã¢â‚¬Å¡Ãƒâ€šÃ‚Â¢ÃƒÆ’Ã†â€™Ãƒâ€šÃ‚Â¢ÃƒÆ’Ã‚Â¢ÃƒÂ¢Ã¢â€šÂ¬Ã…Â¡Ãƒâ€šÃ‚Â¬ÃƒÆ’Ã¢â‚¬Â¦Ãƒâ€šÃ‚Â¡ÃƒÆ’Ã†â€™ÃƒÂ¢Ã¢â€šÂ¬Ã…Â¡ÃƒÆ’Ã¢â‚¬Å¡Ãƒâ€šÃ‚Â¬ÃƒÆ’Ã†â€™Ãƒâ€ Ã¢â‚¬â„¢ÃƒÆ’Ã¢â‚¬Å¡Ãƒâ€šÃ‚Â¢ÃƒÆ’Ã†â€™Ãƒâ€šÃ‚Â¢ÃƒÆ’Ã‚Â¢ÃƒÂ¢Ã¢â‚¬Å¡Ã‚Â¬Ãƒâ€¦Ã‚Â¡ÃƒÆ’Ã¢â‚¬Å¡Ãƒâ€šÃ‚Â¬ÃƒÆ’Ã†â€™ÃƒÂ¢Ã¢â€šÂ¬Ã‚Â¦ÃƒÆ’Ã‚Â¢ÃƒÂ¢Ã¢â‚¬Å¡Ã‚Â¬Ãƒâ€¦Ã¢â‚¬Å“ are superimposed on the effects of gust loads ÃƒÆ’Ã†â€™Ãƒâ€ Ã¢â‚¬â„¢ÃƒÆ’Ã¢â‚¬Â ÃƒÂ¢Ã¢â€šÂ¬Ã¢â€žÂ¢ÃƒÆ’Ã†â€™ÃƒÂ¢Ã¢â€šÂ¬Ã…Â¡ÃƒÆ’Ã¢â‚¬Å¡Ãƒâ€šÃ‚Â¢ÃƒÆ’Ã†â€™Ãƒâ€ Ã¢â‚¬â„¢ÃƒÆ’Ã¢â‚¬Å¡Ãƒâ€šÃ‚Â¢ÃƒÆ’Ã†â€™Ãƒâ€šÃ‚Â¢ÃƒÆ’Ã‚Â¢ÃƒÂ¢Ã¢â€šÂ¬Ã…Â¡Ãƒâ€šÃ‚Â¬ÃƒÆ’Ã¢â‚¬Â¦Ãƒâ€šÃ‚Â¡ÃƒÆ’Ã†â€™ÃƒÂ¢Ã¢â€šÂ¬Ã…Â¡ÃƒÆ’Ã¢â‚¬Å¡Ãƒâ€šÃ‚Â¬ÃƒÆ’Ã†â€™Ãƒâ€ Ã¢â‚¬â„¢ÃƒÆ’Ã¢â‚¬Å¡Ãƒâ€šÃ‚Â¢ÃƒÆ’Ã†â€™Ãƒâ€šÃ‚Â¢ÃƒÆ’Ã‚Â¢ÃƒÂ¢Ã¢â‚¬Å¡Ã‚Â¬Ãƒâ€¦Ã‚Â¡ÃƒÆ’Ã¢â‚¬Å¡Ãƒâ€šÃ‚Â¬ÃƒÆ’Ã†â€™ÃƒÂ¢Ã¢â€šÂ¬Ã‚Â¦ÃƒÆ’Ã‚Â¢ÃƒÂ¢Ã¢â‚¬Å¡Ã‚Â¬Ãƒâ€¦Ã¢â‚¬Å“ or, the gust envelope, blue dashdots.

### Defining features of the manoeuvre envelope

Let us begin in the origin of the diagram with the positive...

#### stall line.

This is the maximum normal load factor that the aircraft is capable of achieving through the deflection of its control surfaces - for any given airspeed, increasing the $C_\mathrm{L}$ and therefore the load factor $n$ is only possible up to this line, where $C_\mathrm{Lmax}$ is reached and stall will follow.

The load factor is simply the ratio of the normal accelerating resultant force acting on the aircraft and its unaccelerated weight: 

$n(V) = \frac{L}{W}=\frac{\frac{1}{2}\rho_0V^2SC_\mathrm{L}}{W}$,

which peaks when the lift coefficient 'maxes out', so $n_\mathrm{max}$ is the maximum achievable lift over the weight:

$n_\mathrm{max}(V) = \frac{L\mathrm{max}(V)}{W} = \frac{\frac{1}{2}\rho_0V^2SC_\mathrm{Lmax}}{W} = \frac{\frac{1}{2}\rho_0V^2C_\mathrm{Lmax}}{W/S}$.


Now, with increasing equivalent airspeed, this maximum achievable load factor increases, but at a value denoted...

#### $V_\mathrm{A}$ or manoeuvre speed

...it reaches the limit load factor permitted by the certification requirements document (e.g., CS-23, paragraph 23.337) - 3.8 in the case of this example. In other words, once you have exceeded the manoeuvre speed, a full deflection of the control surfaces (if they have sufficient authority) may cause the airframe to exceed the limit load factor.

A similar logic applies to the 'negative $g$' stall line (0-G on the diagram).

The stall curves are typically 'closed off' at the 1g (or -1g) point, as in the area to the left of these points  the aircraft is not capable of generating sufficient lift to support its weight (this is literally what a sub-unity load factor means), even in steady, level flight. Somewhat colloquially the corresponding speeds are often referred to as 'the stall speeds' (normal and inverted), although we must place inverted commas around this because, technically, as this very diagram makes plain, you can stall an aeroplane at any speed if you exceed its maximum lift coefficient. 

Other significant speeds that determine the shape of both envelopes are...

#### ...the design cruise speed $V_\mathrm{C}$  and the design dive speed $V_\mathrm{D}$.

While $V_\mathrm{C}$ is, of course, related to the operationally most favourable speed of the aircraft (for example, the speed that maximises its range), this 'certification value' of the cruise speed must be chosen quite carefully because airworthiness requirements, such as CS 23.335 establish a series of quite complicated constraints and relationships between $V_\mathrm{C}$, $V_\mathrm{D}$, wing loading and so on.

When choosing $V_\mathrm{D}$, the designer must also bear in mind the various aeroealsticity-related issues of high speed flight. See, for example, the stipulations in CS 23.629, such as that *'any rational analysis used to predict freedom from flutter, control reversal and divergence must cover all speeds up to 1.2 $V_\mathrm{D}$'*

### Gust loads

Let us now consider the impact of accounting for gust loads.

These are caused by rising or sinking columns of air which the aircraft might penetrate, with the result being a temporary increase or decrease, respectively, of the angle of attack and thus the load factor. The gust-related load factor increases linearly with airspeed, which is why we see gust *lines* on the diagram (emanating from the (0,1) point). 

CS 23.333 requires that we account for positive (up) and negative (down) gusts of 50 fps (or 15.24m/s) at $V_\mathrm{C}$ at altitudes between sea level and 6,096m (20,000ft). Points C and F on the diagram indicate the load factor values calculated for +/-50fps respectively at this speed and this gives us the first pair of gust lines (CS 23.341 tells us how to calculate the gust load factor as a function of airspeed).

Similarly, positive and negative gusts of 25fps at $V_\mathrm{D}$ must be considered at altitudes below 20,000ft, which yields the next pair of gust lines.

Finally, all four lines are trimmed at $V_\mathrm{C}$ and $V_\mathrm{D}$ as appropriate and the end points of the positive pair and the negative pair are connected with straight lines, as per the last paragraph of 23.333.

With all the key constraints and landmark points in place, the complete flight envelope can be drawn up (shown as the shaded area here). This is done by taking the more conservative of the manoeuvre and gust envelopes in each point of the V-n space.

ADRpy does all this construction work automatically, but if you're drawing your own diagram or writing your own code you may wish to look out for a few gotchas. 

An example is the tricky corner around point A, where, if the 50fps gust line does not intersect the stall curve, you have to extend the latter until it does. The abscissa of the intersection point is... 

#### ...$V_\mathrm{B}$, a.k.a, the maximum gust intensity speed

This speed is noteworthy, because above it the strongest expected gust - 50fps in this case, but may have to be assumed as high as 66fps for commuter category aircraft - could cause the aircraft to stall. A hat tip to Snorri Gudmundsson for pointing out in his GA aircraft design book that this 'corner' does not appear in the example diagram shown in Part 23, but may well be a feature of aircraft with low wing loadings.

### Notes

**1.** Not all structural limitations of an airframe are captured in a V-n diagram. Asymmetrical air loads and landing loads are notable exceptions. 

**2.** Some structural limitations seen on aircraft placards are not due to V-n diagram violations. For example flap deployment at high altitudes may be prohibited simply because the manufacturer did not apply for this condition to be permitted (they may not have seen a sufficiently compelling need to certify it).

**3.** The ADRpy implementation of the V-n diagram is currently limited to Part 23 aircraft (CS-23/14 CFR 23). Part 25 and other algorithms may be included in future releases.

**4.** The example discussed here applies to a single altitude - weight - configuration combination (sea level - MTOW - clean). V-n diagrams are typically drawn up for a range of such combinations and different combinations may be critical for different components of the aircraft. For example, the minimum flying weight may be critical for structural components sized for inertial loads, such as engine pylons or various parts of the internal secondary structure (supports, equipment mounts, etc.). 

**5.** We are largely concerned with limit loads here, but structural design also requires fatigue calculations. For these you may need to consider **loading spectra** too - see, for example FAA Advisory Circular **AC 23-13A** 'Fatigue, Fail-Safe, And Damage Tolerance Evaluation Of Metallic Structures For Normal, Utility, Acrobatic, and Commuter Category Airplanes', which sets out acceptable means of showing compliance on this aspect with 14 CFR Part 23.

## Another worked example: twin turboprop commuter 

Let us now consider another example, based on a Beechcraft 1900D-like commuter turboprop (most of the inputs are simply guesses and not based on any proprietary data). 

In [10]:
# designbrief = {}

designdef = {'aspectratio': 10.48, 'wingarea_m2': co.feet22m2(310), 'weight_n': co.lbf2n(17120)}

designperf = {'CLmaxclean': 1.6, 'CLminclean': -1, 'CLslope': 6.28}

designpropulsion = "turboprop" 

designatm = at.Atmosphere() # set the design atmosphere to a zero-offset ISA
    
# Max operating speed: 248KIAS up to 13,200ft, M0.48 above (at 20,000ft M0.48 = 293KTAS = 214KEAS) 
csbrief={'cruisespeed_keas': 204, 'divespeed_keas': 248, 
         'altitude_m': co.feet2m(0),
         'weightfraction': 0.7, 'certcat': 'comm'}

In [11]:
Beech1900Dspec = aw.CertificationSpecifications(designbrief, designdef, designperf, designatm, designpropulsion, csbrief)

In [12]:
points = Beech1900Dspec.flightenvelope(textsize=15, figsize_in=[15, 10], show=True)

AttributeError: 'AircraftConcept' object has no attribute 'clmaxclean'